In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Resample EEG to match EMG's length (if necessary)
if len(eeg_data) != len(emg_data):
    eeg_data = resample(eeg_data, num=len(emg_data))

# Dynamic Time Warping (DTW) for offline alignment
distance, path = fastdtw(eeg_data.values, emg_data.values, dist=euclidean)
aligned_eeg = eeg_data.iloc[[p[0] for p in path]]
aligned_emg = emg_data.iloc[[p[1] for p in path]]

# Combine aligned data
combined_data = pd.concat([aligned_eeg.reset_index(drop=True), aligned_emg.reset_index(drop=True)], axis=1)

# Add noise to the input data
noise_factor = 0.1  # Adjust noise level
combined_data += noise_factor * np.random.normal(size=combined_data.shape)

# Randomly flip a percentage of labels
label_flip_percentage = 0.1  # 10% label noise
num_flips = int(len(labels) * label_flip_percentage)
flip_indices = np.random.choice(len(labels), size=num_flips, replace=False)
labels[flip_indices] = np.random.choice(np.unique(labels), size=num_flips)

# Define a custom Dataset
class EEGEMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Model Definitions (unchanged from earlier code)
class ShallowConvNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ShallowConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(16 * ((input_dim - 2) // 2), num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        return self.fc(x)

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        if x.dim() == 2:  # Reshape if input is flat
            x = x.unsqueeze(1)
        _, (h_n, _) = self.lstm(x)
        h_n = h_n.squeeze(0)
        return self.fc(h_n)

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        if x.dim() == 2:  # Reshape if input is flat
            x = x.unsqueeze(1)
        _, h_n = self.gru(x)
        h_n = h_n.squeeze(0)
        return self.fc(h_n)

class CNNLSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(CNNLSTMNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.lstm = nn.LSTM(16, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.permute(0, 2, 1)
        _, (h_n, _) = self.lstm(x)
        h_n = h_n.squeeze(0)
        return self.fc(h_n)

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim, num_classes):
        super(VAE, self).__init__()
        self.encoder = nn.Linear(input_dim, latent_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)
        self.fc = nn.Linear(latent_dim, num_classes)

    def forward(self, x):
        z = torch.relu(self.encoder(x.view(x.size(0), -1)))
        x_recon = torch.sigmoid(self.decoder(z))
        logits = self.fc(z)
        return logits, x_recon


# Train and Evaluate Models
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
model_classes = {
    "ShallowConvNet": ShallowConvNet,
    "LSTMNet": LSTMNet,
    "GRUNet": GRUNet,
    "CNNLSTMNet": CNNLSTMNet,
    "VAE": VAE,
}
results = {name: [] for name in model_classes.keys()}
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

# Evaluation function with Confusion Matrix
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef

# Updated evaluation function with additional metrics
def evaluate_model(model, val_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            if isinstance(model, VAE):
                logits, _ = model(X_batch)  # VAE returns two outputs
            else:
                logits = model(X_batch)  # Other models return single output

            _, predicted = torch.max(logits, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    print("\nEvaluation Metrics:")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}, Cohen's Kappa: {kappa:.4f}, MCC: {mcc:.4f}")
    print("Confusion Matrix:\n", conf_matrix)

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'balanced_accuracy': balanced_acc,
        'cohens_kappa': kappa,
        'mcc': mcc
    }


# Online Adaptation with Regularization
def online_adaptation_with_regularization(
    model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001
):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)

    for cycle in range(num_cycles):
        print(f"\nOnline Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            optimizer.zero_grad()

            # Handle VAE separately because it returns (logits, x_recon)
            if isinstance(model, VAE):
                outputs, _ = model(X_batch)  # Extract only logits
            else:
                outputs = model(X_batch)  # Other models return only logits
            
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()

        # Evaluate after each adaptation cycle
        print("\nValidation Metrics After Online Adaptation Cycle:")
        evaluate_model(model, val_loader)

online_adaptation_percentage = 0.3  # 30% data for online adaptation

# Training and Evaluation Loop with Online Adaptation
epochs = 10  # Define number of training epochs

for fold, (train_index, val_index) in enumerate(kf.split(combined_data)):
    print(f"\nFold {fold + 1}/{k}")

    X_train, X_val = combined_data.values[train_index], combined_data.values[val_index]
    y_train, y_val = labels[train_index], labels[val_index]

    # Define model parameters
    input_dim = X_train.shape[1]  # Define input dimension dynamically
    hidden_dim = 256  # Define hidden_dim for LSTM, GRU, CNN-LSTM
    latent_dim = 64  # Define latent dimension for VAE
    num_classes = len(np.unique(labels))

    # Split part of training data for online adaptation
    split_idx = int(len(X_train) * online_adaptation_percentage)
    buffer_X, X_train = X_train[:split_idx], X_train[split_idx:]
    buffer_y, y_train = y_train[:split_idx], y_train[split_idx:]

    train_dataset = EEGEMGDataset(X_train, y_train)
    val_dataset = EEGEMGDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    for model_name, ModelClass in model_classes.items():
        print(f"\nTraining {model_name} on Fold {fold + 1}")

        # Define model architecture properly
        if model_name == "VAE":
            model = ModelClass(input_dim=input_dim, latent_dim=latent_dim, num_classes=num_classes)
        elif model_name == "ShallowConvNet":
            model = ModelClass(input_dim=input_dim, num_classes=num_classes)
        else:  # LSTM, GRU, CNN-LSTM need hidden_dim
            model = ModelClass(input_dim=input_dim, hidden_dim=hidden_dim, num_classes=num_classes)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

        for epoch in range(epochs):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()

                if model_name == "VAE":
                    logits, _ = model(X_batch)  # VAE returns both logits and reconstructions
                else:
                    logits = model(X_batch)  # Other models only return logits

                loss = criterion(logits, y_batch)
                loss.backward()
                optimizer.step()

        print("\nValidation Metrics Before Online Adaptation:")
        metrics = evaluate_model(model, val_loader)
        results[model_name].append(metrics)

        # Save trained model for the fold
        torch.save(model.state_dict(), f"{model_name}_fold{fold + 1}.pth")
        print(f"Saved {model_name} for Fold {fold + 1}")

        # Perform online adaptation
        print(f"\nStarting Online Adaptation for {model_name} on Fold {fold + 1}...")
        online_adaptation_with_regularization(model, optimizer, buffer_X, buffer_y, criterion, val_loader)

# Compute Average Metrics
# Compute Average Metrics across folds
for model_name in results:
    avg_metrics = {metric: np.mean([fold[metric] for fold in results[model_name]]) for metric in results[model_name][0]}
    print(f"\n{model_name} - Average Metrics:")
    for metric, value in avg_metrics.items():
        print(f"{metric.replace('_', ' ').capitalize()}: {value:.4f}")




Fold 1/5

Training ShallowConvNet on Fold 1


/home/tipu/.local/lib/python3.10/site-packages/torch/autograd/graph.py:769: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



Validation Metrics Before Online Adaptation:

Evaluation Metrics:
Precision: 0.4885, Recall: 0.5997, F1-Score: 0.5174, Accuracy: 0.5997
Balanced Accuracy: 0.6114, Cohen's Kappa: 0.5344, MCC: 0.5708
Confusion Matrix:
 [[  0   0   7 181   1   3   3]
 [  0   0 118  76   2   5   0]
 [  0   0 160  10   4   1   1]
 [  0   0   6 175   3   4   2]
 [  0   0   3  30 124   3  27]
 [  0   0   1  11   4 169   2]
 [  0   0   5   8   3   6 166]]
Saved ShallowConvNet for Fold 1

Starting Online Adaptation for ShallowConvNet on Fold 1...

Online Adaptation Cycle 1/3

Validation Metrics After Online Adaptation Cycle:

Evaluation Metrics:
Precision: 0.3523, Recall: 0.2825, F1-Score: 0.2016, Accuracy: 0.2825
Balanced Accuracy: 0.2746, Cohen's Kappa: 0.1581, MCC: 0.2249
Confusion Matrix:
 [[189   6   0   0   0   0   0]
 [102  98   0   0   1   0   0]
 [ 39 134   0   0   3   0   0]
 [182   5   0   0   3   0   0]
 [100   3   0   0  84   0   0]
 [179   2   0   0   3   3   0]
 [181   5   0   0   2   0   0]]

O